In [1]:
import re
import json
import requests
import struct
from omero.model.enums import UnitsLength
from omero.model import RoiI, MaskI, ImageI, PolygonI, LengthI
from omero.gateway import BlitzGateway, rint, rdouble, rstring
from getpass import getpass
import omero.clients
from scipy import ndimage
from skimage.segmentation import clear_border
from skimage.measure import regionprops_table
from skimage.color import label2rgb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
from datetime import datetime
import cv2


# csvtocsvw_url='https://csvtocsvw.matolab.org'
# maptomethod_url='https://maptomethod.matolab.org'
# rdfconverter_url='https://rdfconverter.matolab.org'
# using private dev apps for now
csvtocsvw_url = 'http://docker-dev.iwm.fraunhofer.de:5001'
maptomethod_url = 'http://docker-dev.iwm.fraunhofer.de:5002'
rdfconverter_url = 'http://docker-dev.iwm.fraunhofer.de:5003'


def post_request(url, headers, data, files=None):
    try:
        if files:
            # should crate a multipart form upload
            response = requests.post(
                url, data=data, headers=headers, files=files)
        else:
            # a application json post request
            response = requests.post(
                url, data=json.dumps(data), headers=headers)
        response.raise_for_status()

    except requests.exceptions.RequestException as e:
        # placeholder for save file / clean-up
        raise SystemExit(e) from None
    return response


def annotate_csv_uri(csv_url: str, encoding: str = 'auto',):
    # curl -X 'POST' \ 'https://csvtocsvw.matolab.org/api/annotation' \ -H 'accept: application/json' \ -H 'Content-Type: application/json' \ -d '{ "data_url": "https://github.com/Mat-O-Lab/CSVToCSVW/raw/main/examples/example.csv", "separator": "auto", "header_separator": "auto", "encoding": "auto" }'
    url = csvtocsvw_url+"/api/annotate"
    data = {
        "data_url": csv_url,
        "encoding": encoding
    }
    headers = {'Content-type': 'application/json',
               'Accept': 'application/json'}
    r = post_request(url, headers, data).json()
    filename = r['filename']
    file = json.dumps(r['filedata'], indent=4).encode('utf-8')
    print('csvw annotation file created, suggested name: {}'.format(filename))
    with open(filename, "wb") as f:
        f.write(file)
        print('wrote csvw meta data to {}'.format(filename))
    return True



def annotate_csv_upload(filepath: str, encoding: str = 'auto',):
    # curl -X 'POST' \ 'https://csvtocsvw.matolab.org/api/annotate_upload?encoding=auto' \ -H 'accept: application/json' \ -H 'Content-Type: multipart/form-data' \ -F 'file=@detection_runs.csv;type=text/csv'
    url = csvtocsvw_url+"/api/annotate_upload?encoding=auto"
    headers = {"accept": "application/json"}
    head, tail = os.path.split(filepath)
    files = {"file": (tail, open(filepath, "rb"), "text/csv")}
    response = requests.post(url, headers=headers, files=files)
    if response.status_code == 200:
        return response.json()
    else:
        return response


def csvw_to_rdf(meta_url: str, format: str = 'turtle',):
    # curl -X 'POST' \ 'https://csvtocsvw.matolab.org/api/rdf' \ -H 'accept: application/json' \ -H 'Content-Type: application/json' \ -d '{ "metadata_url": "https://github.com/Mat-O-Lab/resources/raw/main/rdfconverter/tests/detection_runs-metadata.json", "format": "turtle" }'
    url = csvtocsvw_url+"/api/rdf"
    data = {
        "metadata_url": meta_url,
        "format": format
    }
    headers = {'Content-type': 'application/json',
               'Accept': 'application/json'}
    #r = requests.post(url, data=json.dumps(data), headers=headers)
    r = post_request(url, headers, data)
    if r.status_code == 200:
        d = r.headers['content-disposition']
        fname = re.findall("filename=(.+)", d)[0]
        with open(fname, 'wb') as f:
            f.write(r.content)
        print('writen serialized table to {}'.format(fname))
        return True
    else:
        return False


def create_mapping(meta_url: str, method_url: str, map_dict: dict, data_super_classes: list, predicate: str, method_super_classes: list):
    # curl -X 'POST' \ 'https://csvtocsvw.matolab.org/api/annotation' \ -H 'accept: application/json' \ -H 'Content-Type: application/json' \ -d '{ "data_url": "https://github.com/Mat-O-Lab/CSVToCSVW/raw/main/examples/example.csv", "separator": "auto", "header_separator": "auto", "encoding": "auto" }'
    url = maptomethod_url+"/api/mapping"
    data = {
        "data_url": meta_url,
        "method_url": method_url,
        "data_super_classes": data_super_classes,
        "predicate": predicate,
        "method_super_classes": method_super_classes,
        "map": map_dict
    }
    headers = {'Content-type': 'application/json',
               'Accept': 'application/json'}
    r = post_request(url, headers, data)
    if r.status_code == 200:
        d = r.headers['content-disposition']
        fname = re.findall("filename=(.+)", d)[0]
        with open(fname, 'wb') as f:
            f.write(r.content)
        print('writen mapping file to {}'.format(fname))
        return True
    else:
        return False


def get_joined_rdf(map_url: str, data_url: str, duplicate_for_table=False):
    # curl -X 'POST' \ 'https://csvtocsvw.matolab.org/api/annotation' \ -H 'accept: application/json' \ -H 'Content-Type: application/json' \ -d '{ "data_url": "https://github.com/Mat-O-Lab/CSVToCSVW/raw/main/examples/example.csv", "separator": "auto", "header_separator": "auto", "encoding": "auto" }'
    url = rdfconverter_url+"/api/createrdf"
    data = {
        "mapping_url": map_url,
        "data_url": data_url,
        "duplicate_for_table": duplicate_for_table
    }
    headers = {'Content-type': 'application/json',
               'Accept': 'application/json'}
    r = post_request(url, headers, data)
    if r.status_code == 200:
        r=r.json()
        filename=r['filename']
        print("applied {} mapping rules and skipped {}".format(r['num_mappings_applied'],r['num_mappings_skipped']))
        with open(filename, "w") as f:
            f.write(r['graph'])
            print('wrote joint graph to {}'.format(filename))
    else:
        return r


# need credentials
username = input("Username: ")
password = getpass("OMERO Password: ")

meta_extractor_api = "https://metadata.omero.matolab.org/api/"

HOST = 'wss://wss.omero.matolab.org'
client = omero.client(HOST)


def test_connect():
    session = client.createSession(username, password)
    with BlitzGateway(client_obj=client) as conn:
        print("Connected as {}".format(conn.getUser().getName()))
        print("User ID: {}".format(conn.getUser().getId()))
        print("User Full Name: {}".format(conn.getUser().getFullName()))

        print("Your Groups:")
        for g in conn.getGroupsMemberOf():
            print("   Name:", g.getName(), " ID:", g.getId())
        group = conn.getGroupFromContext()
        print("Current group: ", group.getName())


def get_datasets():
    session = client.createSession(username, password)
    with BlitzGateway(client_obj=client) as conn:
        # demo project
        project = conn.getObject("Project", 51)
        return list(project.listChildren())

# Load images in a specified dataset method


def get_images(dataset_id):
    """
    Load the images in the specified dataset
    :param conn: The BlitzGateway
    :param dataset_id: The dataset's id
    :return: The Images or None
    """
    session = client.createSession(username, password)
    with BlitzGateway(client_obj=client) as conn:
        dataset = conn.getObject("Dataset", dataset_id)
        images = []
        if dataset:
            for image in dataset.listChildren():
                images.append(image)
            if len(images) == 0:
                return None

        for image in images:
            print("---- Loaded image ID:", image.id)

        # Print dataset ID and name
        print("Dataset ID:", dataset.getId())
        print("Dataset Name:", dataset.getName())
    return images


def create_mask(mask_bytes, bytes_per_pixel=1):
    if bytes_per_pixel == 2:
        divider = 16.0
        format_string = "H"  # Unsigned short
        byte_factor = 0.5
    elif bytes_per_pixel == 1:
        divider = 8.0
        format_string = "B"  # Unsigned char
        byte_factor = 1
    else:
        message = "Format %s not supported"
        raise ValueError(message)
    steps = math.ceil(len(mask_bytes) / divider)
    mask = []
    for i in range(int(steps)):
        binary = mask_bytes[
            i * int(divider):i * int(divider) + int(divider)]
        format = str(int(byte_factor * len(binary))) + format_string
        binary = struct.unpack(format, binary)
        s = ""
        for bit in binary:
            s += str(bit)
        mask.append(int(s, 2))
    return bytearray(mask)

# We have a helper function for creating an ROI and linking it to new shapes


def create_roi(img_id, shapes):
    # create an ROI, link it to Image
    roi = RoiI()
    # use the omero.model.ImageI that underlies the 'image' wrapper
    session = client.createSession(username, password)
    with BlitzGateway(client_obj=client) as conn:
        img = conn.getObject("Image", img_id)
        roi.setImage(ImageI(img_id, False))
        for shape in shapes:
            roi.addShape(shape)
        # Save the ROI (saves any linked shapes too)
        updateService = conn.getUpdateService()
        return updateService.saveAndReturnObject(roi)


def delete_rois(image_id):
    session = client.createSession(username, password)
    with BlitzGateway(client_obj=client) as conn:
        roi_service = conn.getRoiService()
        result = roi_service.findByImage(image_id, None)
        roi_ids = [roi.id.val for roi in result.rois]
        if roi_ids:
            conn.deleteObjects("Roi", roi_ids)
        print("deleted rois: {}".format(roi_ids))


def rgba_to_int(red, green, blue, alpha=255):
    """ Return the color as an Integer in RGBA encoding """
    return int.from_bytes([red, green, blue, alpha],
                          byteorder='big', signed=True)


def print_image_details(img_id):
    # Retrieve information about the image
    session = client.createSession(username, password)
    with BlitzGateway(client_obj=client) as conn:
        image = conn.getObject("Image", img_id)
        print("Image Name:", image.getName())
        print("Image Name:", image.id)
        print("Image Description:", image.getDescription())
        print("Image SizeX:", image.getSizeX())
        print("Image SizeY:", image.getSizeY())
        print("Image SizeZ:", image.getSizeZ())
        print("Image SizeC:", image.getSizeC())
        print("Image SizeT:", image.getSizeT())

        x = image.getName()

        # List Channels (loads the Rendering settings to get channel colors)
        for channel in image.getChannels():
            print('Channel:', channel.getLabel())
            print('Color:', channel.getColor().getRGB())
            print('Lookup table:', channel.getLut())
            print('Is reverse intensity?', channel.isReverseIntensity())

        print(image.countImportedImageFiles())
        file_count = image.countFilesetFiles()
        # list files
        if file_count > 0:
            for orig_file in image.getImportedImageFiles():
                name = orig_file.getName()
                path = orig_file.getPath()
                print(name)
                print(path)
    return x


def get_grayscale(img_id):
    session = client.createSession(username, password)
    with BlitzGateway(client_obj=client) as conn:
        image = conn.getObject("Image", img_id)
        z = image.getSizeZ() / 2
        t = 0
        rendered_image = image.renderImage(z, t)
        image_array = np.asarray(rendered_image)
        # Convert the image to grayscale
        gray = cv2.cvtColor(image_array, cv2.COLOR_RGB2GRAY)
    return gray


def create_omero_roi_polygons(image_id, contours):
    the_t = 0
    the_z = 0
    polygons = []
    # create an ROI with a single polygon
    for i, contour in enumerate(contours):
        polygon = PolygonI()
        polygon.theZ = rint(the_z)
        polygon.theT = rint(the_t)
        color = list(np.random.choice(range(256), size=3))
        polygon.strokeColor = rint(rgba_to_int(color[0], color[1], color[2]))
        polygon.fillColor = rint(rgba_to_int(color[0], color[1], color[2]))

        polygon.strokeWidth = LengthI(1, UnitsLength.PIXEL)
        pts = ["{},{}".format(point[:, 0][0], point[:, 1][0])
               for point in contour]
        pts_list = " ".join(pts)
        polygon.points = rstring(pts_list)
        # print(pts_list[:30])
        polygon.textValue = rstring("Precipitate"+str(i))
        polygons.append(polygon)
    # print(polygon.info())
    # print(polygon.__dir__())
    create_roi(image_id, polygons)
    print("added {} polygon shapes to image".format(len(polygons)))


def run_detection(
        image,
        threshold_method: str = "Otsu",
        size_thresh: float = 80,
        dilate_kernel_size: int = 3,
        median_filter_radius: int = 4,
        save_to_file=False,
        plot=False
):

    name = image.name
    print("Image Name: "+name)

    gray = get_grayscale(image.id)

    print("Grayscale image shape:", gray.shape)
    print("Grayscale image data type:", gray.dtype)

    # Apply median filter using OpenCV

    #selem = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2*median_filter_radius + 1, 2*median_filter_radius+1))
    filtered = cv2.medianBlur(gray, 2 * median_filter_radius + 1)

    # Display the original and filtered images side by side
    if plot:
        fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 5))
        ax0.imshow(gray, cmap='gray')
        ax0.set_title('Step1: Load Original Image')
        ax0.axis('off')
        ax1.imshow(filtered, cmap='gray')
        ax1.set_title('Step2: Median Blur Filter')
        ax1.axis('off')

    # Apply Otsu's thresholding using OpenCV
    if threshold_method == "Otsu":
        ret, thresh = cv2.threshold(
            filtered, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        print('Threshold value is {}'.format(ret))
    else:
        print("only Otsu thresholding supported, skippping thresholding")
        thresh = gray

    # Dilate the thresholded image using a 3x3 kernel
    kernel = np.ones((dilate_kernel_size, dilate_kernel_size), np.uint8)
    dilated = cv2.dilate(thresh, kernel)

    if plot:
        fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 5))
        ax0.imshow(thresh, cmap='gray')
        ax0.set_title('Step3: Otsu Threshold')
        ax0.axis('off')
        ax1.imshow(dilated, cmap='gray')
        ax1.set_title('Step4: Dilated Image')
        ax1.axis('off')

    # Remove small objects using OpenCV's morphologyEx function
    morphed = cv2.morphologyEx(dilated, cv2.MORPH_OPEN, kernel, iterations=1)
    contours, hierarchy = cv2.findContours(
        morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < size_thresh:
            cv2.drawContours(morphed, [cnt], 0, 0, -1)

    # delete all previously defined shapes
    delete_rois(image.id)

    # upload shapes to omero
    create_omero_roi_polygons(image.id, contours)

    # Apply clear border to the dilated image
    mask = morphed == 255
    mask = clear_border(mask)

    # Label the mask and count the number of precipitates detected
    s = [[1, 1, 1], [1, 1, 1], [1, 1, 1]]
    labeled_mask, num_labels = ndimage.label(mask, structure=s)
    img2 = label2rgb(labeled_mask, bg_label=0)

    # Display the labeled mask
    if plot:
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.imshow(img2)
        ax.set_title('Step5: Contour Finding')
        ax.axis('off')
        plt.show()
        print(f"Number of precipitates detected: {num_labels}")

    plt.show()


test_connect()


Connected as thomas.hanke
User ID: 102
User Full Name: Thomas Hanke
Your Groups:
   Name: matolab  ID: 53
   Name: kupferdigital  ID: 54
Current group:  matolab


## The Pipeline

In [2]:
#load all images from selected datasets, run detection algorithm and upload found contours as polygons in a omero roi
#results are saved as detection_runs.csv
datasets=get_datasets()
data=list()
for dataset in datasets:
    print(dataset.name)
    images = get_images(dataset.id)
    for image in images:
        startTime = datetime.now().isoformat()
        info = image.name.split(".",1)[0].split('_')
        state=info[0].split("-")
        if len(state)>1:
            anneal_temp, anneal_time = info[0].split("-")
            anneal_temp = float(anneal_temp.rsplit('C',1)[0])
            anneal_time = float(anneal_time.rsplit('h',1)[0])
        else:
            anneal_temp, anneal_time = 0.0, 0.0
        specimen = info[1].split('Sample',1)[-1]
        pos = info[2].split('Sample',1)[-1]
        threshold_method="Otsu"
        size_thresh=80
        dilate_kernel_size=3
        median_filter_radius=4
        info_dict={
            "SpecimenName": dataset.name+'_'+specimen,
            "Aging Temp [°C]": float(anneal_temp),
            "Aging Time [h]": float(anneal_time),
            "Creep Stress [MPa]": 0,
            "Dataset": meta_extractor_api+"dataset/"+str(dataset.id),
            "Position_Id": pos,
            "Image": meta_extractor_api+"image/"+str(image.id),
            "ROIs": meta_extractor_api+"rois/"+str(image.id),
            "DiskRadiusValue [px]": median_filter_radius,
            "Threshold Method": threshold_method,
            "Dilation Kernel Size [px]": dilate_kernel_size,
            "Date": startTime
        }
        run_detection(image,threshold_method="Otsu",size_thresh=80,dilate_kernel_size=3,median_filter_radius=4)
        data.append(info_dict)
    #     break
    # break
df=pd.DataFrame(data)
df.to_csv('detection_runs.csv')

190C_1000h
---- Loaded image ID: 83
---- Loaded image ID: 84
---- Loaded image ID: 85
---- Loaded image ID: 74
---- Loaded image ID: 75
---- Loaded image ID: 76
---- Loaded image ID: 77
---- Loaded image ID: 78
---- Loaded image ID: 79
---- Loaded image ID: 80
---- Loaded image ID: 81
---- Loaded image ID: 82
Dataset ID: 53
Dataset Name: 190C_1000h
190C_25000h_S1
---- Loaded image ID: 142
---- Loaded image ID: 143
---- Loaded image ID: 133
---- Loaded image ID: 134
---- Loaded image ID: 135
---- Loaded image ID: 136
---- Loaded image ID: 137
---- Loaded image ID: 138
---- Loaded image ID: 139
---- Loaded image ID: 140
---- Loaded image ID: 141
Dataset ID: 58
Dataset Name: 190C_25000h_S1
190C_25000h_S2
---- Loaded image ID: 153
---- Loaded image ID: 144
---- Loaded image ID: 145
---- Loaded image ID: 146
---- Loaded image ID: 147
---- Loaded image ID: 148
---- Loaded image ID: 149
---- Loaded image ID: 150
---- Loaded image ID: 151
---- Loaded image ID: 152
Dataset ID: 59
Dataset Name: 

# Processing Metadata

In [4]:
# annotate detection_runs.csv
response=annotate_csv_uri("https://github.com/BAMresearch/DF-TEM-PAW/raw/main/detection_runs.csv")

csvw annotation file created, suggested name: detection_runs-metadata.json
wrote csvw meta data to detection_runs-metadata.json


In [5]:
# serialize table to rdf, uses already commited files on main branch
meta_url="https://github.com/BAMresearch/DF-TEM-PAW/raw/main/detection_runs-metadata.json"
response=csvw_to_rdf(meta_url)

writen serialized table to detection_runs.ttl


In [221]:
# create a rule bases mapping between the data in detection_runs and the precipitate analysis knowledge graph 
meta_url="https://github.com/BAMresearch/DF-TEM-PAW/raw/main/detection_runs-metadata.json"
method_url="https://github.com/BAMresearch/DF-TEM-PAW/raw/main/PrecipitateAnalysisWorkflow.ttl"
d_classes= [
    "http://www.w3.org/ns/oa#Annotation",''
    "http://www.w3.org/ns/csvw#Column"
]
m_classes=["https://w3id.org/pmd/co/Metadata",]
pred="https://w3id.org/pmd/co/isResourceOf"
map_dict={
    "diskRadius_": "table-1-DiskradiusvaluePx",
    "kernel_": "table-1-DilationKernelSizePx",
    "thresholdMethod_": "table-1-ThresholdMethod",
    "specimenAgingTemperature_": "table-1-AgingTempC",
    "specimenAgingTime_": "table-1-DiskradiusvaluePx",
    "specimenCreepStess_": "table-1-CreepStressMpa",
    "specimenName_": "table-1-Specimenname",
    "specimenAgingTime_": "table-1-AgingTimeH",
    "imageDataset_":"table-1-Dataset",
    "darkfieldTransmissionElectronMicroscopeImage_": "table-1-Image",
}
create_mapping(meta_url=meta_url,method_url=method_url,data_super_classes=d_classes,predicate=pred,method_super_classes=m_classes,map_dict=map_dict)

writen mapping file to detection_runs-map.yaml


True

In [227]:
# join all data and replicate template knowledge graph for every row in table
mapping_url = "https://github.com/BAMresearch/DF-TEM-PAW/raw/main/detection_runs-map.yaml"
data_url = "https://github.com/BAMresearch/DF-TEM-PAW/raw/main/detection_runs.ttl"
duplicate_for_table = True
get_joined_rdf(map_url=mapping_url,data_url=data_url,duplicate_for_table=duplicate_for_table)

applied 7 mapping rules and skipped 0
wrote joint graph to detection_runs-joined.ttl
